Материалы для работы:
* https://habr.com/ru/articles/768562/
* https://huggingface.co/learn/audio-course/ru/chapter5/asr_models

# Решение задачи для аудио на Английском

In [2]:
import torch
from transformers import pipeline

# Скачиваем модель для распознавания аудио
# Medium model работает только с английским языком, если необходимо использовать другой язык, то нужно скачать large модель
device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    "automatic-speech-recognition", model="openai/whisper-medium", device=device
)

In [3]:
import os
import torchaudio
from torchaudio.datasets import LIBRISPEECH
import shutil
import warnings
import difflib

warnings.filterwarnings("ignore")

In [4]:
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

#  Скачиваем 1 инстанс датасета для теста нашей модели
try:
    dataset = LIBRISPEECH(data_dir, url="test-clean", download=True)
except Exception as e:
    print(f"Ошибка: {e}")
    exit(1)

sample = dataset[0]
waveform, sample_rate, utterance, speaker_id, chapter_id, utterance_id = sample

# Определяем директорию для сохранения аудиофайла
output_dir = "../audio"
os.makedirs(output_dir, exist_ok=True)

# Сохраняем аудиофайл в формате WAV
torchaudio.save(os.path.join(output_dir, "test_audio.wav"), waveform, sample_rate)

# Сохраняем текстовый файл с транскрипцией аудиофайла
with open(os.path.join(output_dir, "test_audio.txt"), "w") as f:
    f.write(utterance)

# Удаляем директорию с данными для сохранения памяти
shutil.rmtree(data_dir)
print("Загрузка данных завершена")

100%|██████████| 331M/331M [00:44<00:00, 7.80MB/s] 


Test audio WAV file and corresponding text file created successfully.


In [5]:
warnings.filterwarnings("ignore")

# Делаем транскрипцию аудиофайла
result = pipe('../audio/test_audio.wav', max_new_tokens=256)

# Выполняем сравнение транскрипции с оригинальным текстом
transcribed_text = result["text"].lower()
print(f"Транскрипция: {transcribed_text}")

with open('../audio/test_audio.txt') as file:
    original_text = file.read().lower()
print(f"Оригинальный текст: {original_text}")

# Подсчитываем коэффициент сходства
similarity_ratio = difflib.SequenceMatcher(None, transcribed_text, original_text).ratio()

print("Коэффициет сходства:", round(similarity_ratio*100,3), "%")

Транскрипция:  he hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton-pieces to be ladled out in thick, peppered, flour-fattened sauce.
Оригинальный текст: he hoped there would be stew for dinner turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick peppered flour fattened sauce
Коэффициет сходства: 97.196 %


# Решение задачи для аудио на Русском языке